# Import Packages

In [1]:
import os
import numpy as np
import pandas as pd
import csv
from scipy import stats
from scipy.special import inv_boxcox
import sklearn.metrics as met
import statsmodels.api as sm
from sklearn.model_selection import train_test_split, LeaveOneOut
from sklearn.linear_model import Lasso, Ridge, LinearRegression, LogisticRegression
from sklearn.kernel_ridge import KernelRidge

import itertools
import multiprocessing
import matplotlib.pyplot as plt

from datetime import datetime

from numpy.linalg import inv
from sklearn.utils import shuffle

import timeit

## Expand viewing capability

In [2]:
pd.set_option('display.max_rows', 5000)
pd.set_option('display.max_columns', 5000)
pd.set_option('display.width', 1000)

# Import, Restructure and Explore Data

In [3]:
#upload data from PTN csv
PTNdata1Path = ('/Users/nickcforrest/Documents/My Documents/Education/AFIT/AFIT WORK/Thesis Work/Data/PTN_raw_data_clean.csv')
PTNdata1 = pd.read_csv(PTNdata1Path)
PTNdata1 = PTNdata1.iloc[0:, 1:] #get rid of row just counting overall observation

#Rename poorly named columns
PTNdata1.rename(columns={'Maneuver':'Flight/Sim #', 'Training Day':'Training Date'}, inplace=True)
#delete space from Cross-Check column name
PTNdata1.rename(columns = {'Cross-Check ':'Cross-Check'}, inplace = True)

#restructure the training dates
restruct1_start_time= timeit.default_timer() #start time

for i in range(0,len(PTNdata1)):
    PTNdata1['Training Date'][i] = str(PTNdata1['Training Date'][i]) + str('-2018')
    PTNdata1['Training Date'][i] = datetime.strptime(PTNdata1['Training Date'][i], '%d-%b-%Y').strftime('%Y-%m-%d')
restruct1_elapsed_time = timeit.default_timer() - restruct1_start_time  #end time
    
    
#Get ride of any botched flights/ half flights due to weather or something
PTNdata1[~PTNdata1['Flight/Sim #'].str.contains('.')]

#add Training Date column
PTNdata1['Training Date'] = pd.to_datetime(PTNdata1['Training Date'])

#sort all flights by student and by ascending training date
PTNdata1 = PTNdata1.sort_values(by=['Student', 'Training Date'])
print(type(PTNdata1['Training Date'][1]))

print('Time to restructure dates ', restruct1_elapsed_time, ' seconds')

PTNdata1= PTNdata1.rename({'Student':'Student ID',"Turning Rejoin.1":'4-Ship Turning Rejoin', "Straight Ahead Rejoin.1":"4-Ship Straight Ahead Rejoin"}, axis=1)
PTNdata1.drop('Flight/Sim #', axis=1).sort_values(by = 'Training Date').head(20)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app


<class 'pandas._libs.tslibs.timestamps.Timestamp'>
Time to restructure dates  0.37804703800000006  seconds


,Student ID,Training Date,Mission Analysis/Products,Ground Ops,Takeoff,Departure,Basic Aircraft Control,Cross-Check,Enroute Descent / Recovery,Inflight Checks,Inflight Planning,Clearing / Visual Lookout,Communication,Risk Mgmt / Decision Making,Situational Awareness,Task Management,Emergency Procedures,General Knowledge,PATTERNS,Overhead/Closed Pattern,Visual St-In,Landing,No-Flap Landing,Go-Around,Emergency Landing Pattern,CONTACT,G-Awareness,TP Stalls,Slow Flight,Power On Stalls,Contact Recoveries,Spin Recovery,Aileron Roll,Barrel Roll,Pitchback / Sliceback,Cloverleaf,Cuban Eight,Immelmann,Lazy Eight,Loop,Split S,INSTRUMENTS,Vertical S,Unusual Attitudes,Steep Turns,Intercept / Maintain Arc,Fix to Fix,Holding,Full Procedure Approach,Non-Precision Final,Precision Final,Circling Approach,Missed Approach,Night Landing,BASIC FORMATION,Wing Takeoff,Interval Takeoff,Instrument Trail,G-Warmup / Awareness,Lead Platform,Pitchout (Both),Fingertip (Wing),Route (Wing),Fighting Wing (Wing),Straight Ahead Rejoin,Turning Rejoin,Overshoot,Echelon (Wing),Breakout (Wing),Lost Wingman (Both),Extended Trail (Wing),Position Change,Formation Approach (Both),Formation Landing (Both),Battle Damage Check,Flt Integrity / Wingman Consideration,TACTICAL FORMATION,Delay 90,Delay 45,Hook Turn,Shackle,Cross Turn,Fluid Turn,Tactical Rejoins,Fluid Maneuvering,Tac Initial,LOW LEVEL,Course Mx,Course Entry,Time Control,Altitude Control,Checkpoint ID,LL GPS Integration,Tactical Maneuvering,LL Lead Change,4 SHIP FORMATION,Four Ship Admin,Fluid 4,Box Formation,Offset Box,Wall,4-Ship Fingertip,4-Ship Straight Ahead Rejoin,4-Ship Turning Rejoin,CAF INTRO,Heat to Guns Setup,Heat to Guns Maneuvering,Fuel Awareness/Management,Advanced Handling,Perch Setups,Maneuver Selection,Offensive Fighter Mnvr Exec,Defensive Fighter Mnvr Exec,CZ Recognition,Air to Air Weapons Employ,HA Lead Turn Exercise,HA Butterfly Setups,HA BFM Flt Analysis,SA Conventional Range,SA Tactical Range Proc,SA Safe-Excape Maneuver,SA Threat Reaction,SA Weapons Employment,Air to Ground Error Analysis,TACS/JFIRE Procedures,Air to Gnd 2-Ship Mutual Supt,MAF INTRO,Mission Management,VFR Arrival,Tanker Procedures,Reciever Procedures,Airdrop Procedures,Crew Coordination,Single Engine Approach,Single Engine GA/Missed Appch,A/R Overrun,A/R Breakaway,FD/AP Operations,FMS Operations,Device
1377,17,2018-03-03,F,NaN,F,NaN,F,U,F,U,U,NaN,F,U,U,U,NaN,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,F,F,NaN,NaN,NaN,NaN,F,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S
92,1,2018-04-02,U,F,F,U,F,U,F,U,U,F,U,U,U,U,F,F,NaN,F,F,F,NaN,U,NaN,NaN,NaN,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S
788,10,2018-04-02,F,F,F,U,U,U,F,U,U,F,F,F,F,F,F,F,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,F,F,F,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S
1311,16,2018-04-02,F,F,F,U,U,U,U,U,U,F,U,F,U,U,F,F,NaN,U,U,U,NaN,U,NaN,N

# Find incomplete records and dispose

In [4]:
duplicateRowsDF = pd.concat(g for _, g in PTNdata1.groupby(["Student ID","Training Date"]) if len(g) > 1)
#duplicateRowsDF


found that indicies 156,151,292,406,571,674,1217 all seemed to be misentries of data. they all had less than 5 graded events occuring and seemed to be unfinished duplicates of events occuring on the same training date. Also, 1277,254,807,760,1109 were all obviously incomplete.

In [5]:
#toss incompletes records
PTNdata1 = PTNdata1.drop([156,151,292,406,571,674,1217,1277,254,807,760,1109])

In [6]:
#code to populate Training Exercise column
calcEvalDay_start_time= timeit.default_timer()
prevStudent = int(0) #just initializes previous student to 0 so that no other students will be the same.
PTNdata1['Training Exercise'] = [0]*len(PTNdata1) #creates new columns for evaluation number
j = 1
for i in range(0, len(PTNdata1)):
    
    currentStudent = PTNdata1['Student ID'].iloc[i] #identifies student in ith row
    
    if currentStudent == prevStudent:
        j = j + 1
        PTNdata1['Training Exercise'].iloc[i] = j  #records as next evaluation for student
    else:
        j = 1
        PTNdata1['Training Exercise'].iloc[i] = j #if new student, restart back at 1st evaluation day and count up
    
    prevStudent = PTNdata1['Student ID'].iloc[i]
calcEvalDay_elapsed_time = timeit.default_timer() - calcEvalDay_start_time  
print('time to calulate eval day times: ' , calcEvalDay_elapsed_time, ' seconds')



/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:205: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)


time to calulate eval day times:  0.3774970670000002  seconds


In [7]:
PTNdata1[(PTNdata1['Student ID'] == 2)&(PTNdata1['Training Exercise'] == 20)]

,Student ID,Training Date,Flight/Sim #,Mission Analysis/Products,Ground Ops,Takeoff,Departure,Basic Aircraft Control,Cross-Check,Enroute Descent / Recovery,Inflight Checks,Inflight Planning,Clearing / Visual Lookout,Communication,Risk Mgmt / Decision Making,Situational Awareness,Task Management,Emergency Procedures,General Knowledge,PATTERNS,Overhead/Closed Pattern,Visual St-In,Landing,No-Flap Landing,Go-Around,Emergency Landing Pattern,CONTACT,G-Awareness,TP Stalls,Slow Flight,Power On Stalls,Contact Recoveries,Spin Recovery,Aileron Roll,Barrel Roll,Pitchback / Sliceback,Cloverleaf,Cuban Eight,Immelmann,Lazy Eight,Loop,Split S,INSTRUMENTS,Vertical S,Unusual Attitudes,Steep Turns,Intercept / Maintain Arc,Fix to Fix,Holding,Full Procedure Approach,Non-Precision Final,Precision Final,Circling Approach,Missed Approach,Night Landing,BASIC FORMATION,Wing Takeoff,Interval Takeoff,Instrument Trail,G-Warmup / Awareness,Lead Platform,Pitchout (Both),Fingertip (Wing),Route (Wing),Fighting Wing (Wing),Straight Ahead Rejoin,Turning Rejoin,Overshoot,Echelon (Wing),Breakout (Wing),Lost Wingman (Both),Extended Trail (Wing),Position Change,Formation Approach (Both),Formation Landing (Both),Battle Damage Check,Flt Integrity / Wingman Consideration,TACTICAL FORMATION,Delay 90,Delay 45,Hook Turn,Shackle,Cross Turn,Fluid Turn,Tactical Rejoins,Fluid Maneuvering,Tac Initial,LOW LEVEL,Course Mx,Course Entry,Time Control,Altitude Control,Checkpoint ID,LL GPS Integration,Tactical Maneuvering,LL Lead Change,4 SHIP FORMATION,Four Ship Admin,Fluid 4,Box Formation,Offset Box,Wall,4-Ship Fingertip,4-Ship Straight Ahead Rejoin,4-Ship Turning Rejoin,CAF INTRO,Heat to Guns Setup,Heat to Guns Maneuvering,Fuel Awareness/Management,Advanced Handling,Perch Setups,Maneuver Selection,Offensive Fighter Mnvr Exec,Defensive Fighter Mnvr Exec,CZ Recognition,Air to Air Weapons Employ,HA Lead Turn Exercise,HA Butterfly Setups,HA BFM Flt Analysis,SA Conventional Range,SA Tactical Range Proc,SA Safe-Excape Maneuver,SA Threat Reaction,SA Weapons Employment,Air to Ground Error Analysis,TACS/JFIRE Procedures,Air to Gnd 2-Ship Mutual Supt,MAF INTRO,Mission Management,VFR Arrival,Tanker Procedures,Reciever Procedures,Airdrop Procedures,Crew Coordination,Single Engine Approach,Single Engine GA/Missed Appch,A/R Overrun,A/R Breakaway,FD/AP Operations,FMS Operations,Device,Training Exercise
150,2,2018-04-27,Sim #14,G,G,G,G,G,G,G,G,G,G,F,G,G,G,G,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,G,NaN,NaN,NaN,G,NaN,G,G,NaN,G,NaN,G,G,G,G,G,G,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,20


In [8]:
#prints types of data in each column
PTNdata1.dtypes

Student ID                                        int64
Training Date                            datetime64[ns]
Flight/Sim #                                     object
Mission Analysis/Products                        object
Ground Ops                                       object
Takeoff                                          object
Departure                                        object
Basic Aircraft Control                           object
Cross-Check                                      object
Enroute Descent / Recovery                       object
Inflight Checks                                  object
Inflight Planning                                object
Clearing / Visual Lookout                        object
Communication                                    object
Risk Mgmt / Decision Making                      object
Situational Awareness                            object
Task Management                                  object
Emergency Procedures                            

In [9]:
#split maneuvers into categories and get rid of columns that are just category names

basicManeuverNames = ['Mission Analysis/Products', 'Ground Ops','Takeoff','Departure','Basic Aircraft Control','Cross-Check','Enroute Descent / Recovery','Inflight Checks','Inflight Planning','Clearing / Visual Lookout','Communication','Risk Mgmt / Decision Making','Situational Awareness','Task Management','Emergency Procedures','General Knowledge']
patternsNames = ['Overhead/Closed Pattern', 'Visual St-In', 'Landing', 'No-Flap Landing', 'Go-Around', 'Emergency Landing Pattern']
contactNames = ['G-Awareness', 'TP Stalls', 'Slow Flight', 'Power On Stalls', 'Contact Recoveries', 'Spin Recovery', 'Aileron Roll', 'Barrel Roll', 'Pitchback / Sliceback', 'Cloverleaf', 'Cuban Eight', 'Immelmann', 'Lazy Eight', 'Loop', 'Split S']
instrumentNames =['Vertical S', 'Unusual Attitudes', 'Steep Turns', 'Intercept / Maintain Arc', 'Fix to Fix', 'Holding', 'Full Procedure Approach', 'Non-Precision Final', 'Precision Final', 'Circling Approach', 'Missed Approach', 'Night Landing']
basicFormationNames = ['Wing Takeoff', 'Interval Takeoff', 'Instrument Trail', 'G-Warmup / Awareness', 'Lead Platform', 'Pitchout (Both)', 'Fingertip (Wing)', 'Route (Wing)', 'Fighting Wing (Wing)', 'Straight Ahead Rejoin', 'Turning Rejoin', 'Overshoot', 'Echelon (Wing)', 'Breakout (Wing)', 'Lost Wingman (Both)', 'Extended Trail (Wing)', 'Position Change', 'Formation Approach (Both)', 'Formation Landing (Both)', 'Battle Damage Check', 'Flt Integrity / Wingman Consideration']
tacticalFormationNames =['Delay 90', 'Delay 45', 'Hook Turn', 'Shackle', 'Cross Turn', 'Fluid Turn', 'Tactical Rejoins', 'Fluid Maneuvering', 'Tac Initial']
lowLevelNames = ['Course Mx', 'Course Entry', 'Time Control', 'Altitude Control', 'Checkpoint ID', 'LL GPS Integration', 'Tactical Maneuvering', 'LL Lead Change']
fourShipFormationNames = ['Four Ship Admin', 'Fluid 4', 'Box Formation', 'Offset Box', 'Wall', '4-Ship Fingertip']
cafIntroNames = ['Heat to Guns Setup', 'Heat to Guns Maneuvering ', 'Fuel Awareness/Management', 'Advanced Handling', 'Perch Setups', 'Maneuver Selection', 'Offensive Fighter Mnvr Exec', 'Defensive Fighter Mnvr Exec', 'CZ Recognition', 'Air to Air Weapons Employ', 'HA Lead Turn Exercise', 'HA Butterfly Setups', 'HA BFM Flt Analysis', 'SA Conventional Range', 'SA Tactical Range Proc', 'SA Safe-Excape Maneuver', 'SA Threat Reaction', 'SA Weapons Employment', 'Air to Ground Error Analysis', 'TACS/JFIRE Procedures', 'Air to Gnd 2-Ship Mutual Supt']
mafIntroNames = ['Mission Management', 'VFR Arrival', 'Tanker Procedures', 'Reciever Procedures', 'Airdrop Procedures', 'Crew Coordination', 'Single Engine Approach', 'Single Engine GA/Missed Appch', 'A/R Overrun', 'A/R Breakaway', 'FD/AP Operations', 'FMS Operations']

ManueverCategories = ['PATTERNS','CONTACT','INSTRUMENTS','BASIC FORMATION', 'TACTICAL FORMATION', 'LOW LEVEL', '4 SHIP FORMATION', 'CAF INTRO', 'MAF INTRO' ]
PTNdata1 = PTNdata1.drop(ManueverCategories,axis = 'columns')
print(len(PTNdata1.columns))
print(PTNdata1.columns)

133
Index(['Student ID', 'Training Date', 'Flight/Sim #', 'Mission Analysis/Products', 'Ground Ops', 'Takeoff', 'Departure', 'Basic Aircraft Control', 'Cross-Check', 'Enroute Descent / Recovery',
       ...
       'Airdrop Procedures', 'Crew Coordination', 'Single Engine Approach', 'Single Engine GA/Missed Appch', 'A/R Overrun', 'A/R Breakaway', 'FD/AP Operations', 'FMS Operations', 'Device', 'Training Exercise'], dtype='object', length=133)


In [10]:
#convert data to numeric

#replace empty values with NaN and replace NG with value 0. 
PTNdata1.iloc[:,:-2] = PTNdata1.iloc[:,:-2].replace(['',' ','NG','U','F','G','E'], [np.nan,np.nan, np.float64(0), np.float64(1),np.float64(2),np.float64(3),np.float64(4)])


#drop CAF and MAF tracks to standardize
#PTNdata1 = PTNdata1.drop(cafIntroNames+mafIntroNames,axis = 'columns')
#PTNdata1.head()

# Calculating the total MIF for each student at each training exercise

In [11]:
PTNdata1.head()

,Student ID,Training Date,Flight/Sim #,Mission Analysis/Products,Ground Ops,Takeoff,Departure,Basic Aircraft Control,Cross-Check,Enroute Descent / Recovery,Inflight Checks,Inflight Planning,Clearing / Visual Lookout,Communication,Risk Mgmt / Decision Making,Situational Awareness,Task Management,Emergency Procedures,General Knowledge,Overhead/Closed Pattern,Visual St-In,Landing,No-Flap Landing,Go-Around,Emergency Landing Pattern,G-Awareness,TP Stalls,Slow Flight,Power On Stalls,Contact Recoveries,Spin Recovery,Aileron Roll,Barrel Roll,Pitchback / Sliceback,Cloverleaf,Cuban Eight,Immelmann,Lazy Eight,Loop,Split S,Vertical S,Unusual Attitudes,Steep Turns,Intercept / Maintain Arc,Fix to Fix,Holding,Full Procedure Approach,Non-Precision Final,Precision Final,Circling Approach,Missed Approach,Night Landing,Wing Takeoff,Interval Takeoff,Instrument Trail,G-Warmup / Awareness,Lead Platform,Pitchout (Both),Fingertip (Wing),Route (Wing),Fighting Wing (Wing),Straight Ahead Rejoin,Turning Rejoin,Overshoot,Echelon (Wing),Breakout (Wing),Lost Wingman (Both),Extended Trail (Wing),Position Change,Formation Approach (Both),Formation Landing (Both),Battle Damage Check,Flt Integrity / Wingman Consideration,Delay 90,Delay 45,Hook Turn,Shackle,Cross Turn,Fluid Turn,Tactical Rejoins,Fluid Maneuvering,Tac Initial,Course Mx,Course Entry,Time Control,Altitude Control,Checkpoint ID,LL GPS Integration,Tactical Maneuvering,LL Lead Change,Four Ship Admin,Fluid 4,Box Formation,Offset Box,Wall,4-Ship Fingertip,4-Ship Straight Ahead Rejoin,4-Ship Turning Rejoin,Heat to Guns Setup,Heat to Guns Maneuvering,Fuel Awareness/Management,Advanced Handling,Perch Setups,Maneuver Selection,Offensive Fighter Mnvr Exec,Defensive Fighter Mnvr Exec,CZ Recognition,Air to Air Weapons Employ,HA Lead Turn Exercise,HA Butterfly Setups,HA BFM Flt Analysis,SA Conventional Range,SA Tactical Range Proc,SA Safe-Excape Maneuver,SA Threat Reaction,SA Weapons Employment,Air to Ground Error Analysis,TACS/JFIRE Procedures,Air to Gnd 2-Ship Mutual Supt,Mission Management,VFR Arrival,Tanker Procedures,Reciever Procedures,Airdrop Procedures,Crew Coordination,Single Engine Approach,Single Engine GA/Missed Appch,A/R Overrun,A/R Breakaway,FD/AP Operations,FMS Operations,Device,Training Exercise
92,1,2018-04-02,Sim #2,1.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,2.0,2.0,2.0,2.0,2.0,NaN,1.0,NaN,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,1
91,1,2018-04-03,Sim #3,2.0,2.0,2.0,2.0,2.0,1.0,2.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,2.0,2.0,NaN,NaN,2.0,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S,2
90,1,2018-04-04,Flight #1,1.0,2.0,1.0,1.0,1.0,1.0,2.0,1.0,1.0,2.0,1.0,1.0,1.0,1.0,1.0,1.0,NaN,1.0,1.0,NaN,1.0,NaN,2.0,2.0,0.0,2.0,NaN,NaN,2.0,0.0,0.0,1.0,1.0,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,F,3
88,1,2018-04-06,Sim #5,2.0,2.0,3.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0,1.0,2.0,2.0,2.0,3.0,3.0,NaN,3.0,NaN,NaN,NaN,2.

In [12]:
########## write function for calculating MIFS and create MIF column for each student ########################

# total students in class
totalStudents = len(PTNdata1['Student ID'].unique())

#create new column for total MIF
PTNdata1['Total MIF'] = [0]*len(PTNdata1)

# ##########################################################################
# ####################### turn this into function ###########################
# ##########################################################################
#create individual dataframe for every student
for numStud in range(1,totalStudents+1):
    value = PTNdata1[PTNdata1['Student ID'] == numStud]
    exec ("student%s=value" % (numStud))
print(type(student1))

# #calculate MIFS and create MIF column for each student
# ind = 0 
# for i in student2.index:
#     MIF=0
#     for col in student2.columns :
#         if type(student2[col][i]) == np.float64 or type(student2[col][i]) == float:
#             if any(student2[col][0:ind+1].notnull()): 
#                 MIF = MIF + int(np.nanmax(student2[col][0:i+1].values))
#                 #print(col, MIF)
                
#     student2.iloc[ind,-1] = MIF
#     ind = ind + 1
# print(2)

# ##############################################################################
# ##############################################################################

# def calc_total_MIF(numStud):
    

<class 'pandas.core.frame.DataFrame'>


In [13]:
###################################### replace this with cleaner code ############################################
#create mif column for each student

for i in range(0,len(student1)):
    MIF=0
    for col in student1.columns :
        if type(student1[col][i]) == np.float64 or type(student1[col][i]) == float:
            if any(student1[col][0:i+1].notnull()): 
                MIF = MIF + int(np.nanmax(student1[col][0:i+1].values))
                #print(col, MIF)

    student1.iloc[i,-1] = MIF
print(1) 
######################################################################################################   
#for row in range(0,len(student2)):
ind = 0 
for i in student2.index:
    MIF=0
    for col in student2.columns :
        if type(student2[col][i]) == np.float64 or type(student2[col][i]) == float:
            if any(student2[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student2[col][0:i+1].values))
                #print(col, MIF)
                
    student2.iloc[ind,-1] = MIF
    ind = ind + 1

print(2)   

######################################################################################################   
ind = 0 
for i in student3.index:
    MIF=0
    for col in student3.columns :
        if type(student3[col][i]) == np.float64 or type(student3[col][i]) == float:
            if any(student3[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student3[col][0:i+1].values))
                #print(col, MIF)
                
    student3.iloc[ind,-1] = MIF
    ind = ind + 1

print(3)     
######################################################################################################   
ind = 0 
for i in student4.index:
    MIF=0
    for col in student4.columns :
        if type(student4[col][i]) == np.float64 or type(student4[col][i]) == float:
            if any(student4[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student4[col][0:i+1].values))
                #print(col, MIF)
                
    student4.iloc[ind,-1] = MIF
    ind = ind + 1

print(4)    
######################################################################################################   
ind = 0 
for i in student5.index:
    MIF=0
    for col in student5.columns :
        if type(student5[col][i]) == np.float64 or type(student5[col][i]) == float:
            if any(student5[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student5[col][0:i+1].values))
                #print(col, MIF)
                
    student5.iloc[ind,-1] = MIF
    ind = ind + 1
print(5)    
######################################################################################################   
ind = 0 
for i in student6.index:
    MIF=0
    for col in student6.columns :
        if type(student6[col][i]) == np.float64 or type(student6[col][i]) == float:
            if any(student6[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student6[col][0:i+1].values))
                #print(col, MIF)
                
    student6.iloc[ind,-1] = MIF
    ind = ind + 1
print(6)     
######################################################################################################   
ind = 0 
for i in student7.index:
    MIF=0
    for col in student7.columns :
        if type(student7[col][i]) == np.float64 or type(student7[col][i]) == float:
            if any(student7[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student7[col][0:i+1].values))
                #print(col, MIF)
                
    student7.iloc[ind,-1] = MIF
    ind = ind + 1

print(7)     
######################################################################################################   
ind = 0 
for i in student8.index:
    MIF=0
    for col in student8.columns :
        if type(student8[col][i]) == np.float64 or type(student8[col][i]) == float:
            if any(student8[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student8[col][0:i+1].values))
                #print(col, MIF)
                
    student8.iloc[ind,-1] = MIF
    ind = ind + 1
print(8)    
######################################################################################################   
ind = 0 
for i in student9.index:
    MIF=0
    for col in student9.columns :
        if type(student9[col][i]) == np.float64 or type(student9[col][i]) == float:
            if any(student9[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student9[col][0:i+1].values))
                #print(col, MIF)
                
    student9.iloc[ind,-1] = MIF
    ind = ind + 1
print(9)     
######################################################################################################   
ind = 0 
for i in student10.index:
    MIF=0
    for col in student10.columns :
        if type(student10[col][i]) == np.float64 or type(student10[col][i]) == float:
            if any(student10[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student10[col][0:i+1].values))
                #print(col, MIF)
                
    student10.iloc[ind,-1] = MIF
    ind = ind + 1
print(10)     
######################################################################################################   
ind = 0 
for i in student11.index:
    MIF=0
    for col in student11.columns :
        if type(student11[col][i]) == np.float64 or type(student11[col][i]) == float:
            if any(student11[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student11[col][0:i+1].values))
                #print(col, MIF)
                
    student11.iloc[ind,-1] = MIF
    ind = ind + 1
print(11)     
######################################################################################################   
ind = 0 
for i in student12.index:
    MIF=0
    for col in student12.columns :
        if type(student12[col][i]) == np.float64 or type(student12[col][i]) == float:
            if any(student12[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student12[col][0:i+1].values))
                #print(col, MIF)
                
    student12.iloc[ind,-1] = MIF
    ind = ind + 1
print(12)     
######################################################################################################   
ind = 0 
for i in student13.index:
    MIF=0
    for col in student13.columns :
        if type(student13[col][i]) == np.float64 or type(student13[col][i]) == float:
            if any(student13[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student13[col][0:i+1].values))
                #print(col, MIF)
                
    student13.iloc[ind,-1] = MIF
    ind = ind + 1
print(13)     
######################################################################################################   
ind = 0 
for i in student14.index:
    MIF=0
    for col in student14.columns :
        if type(student14[col][i]) == np.float64 or type(student14[col][i]) == float:
            if any(student14[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student14[col][0:i+1].values))
                #print(col, MIF)
                
    student14.iloc[ind,-1] = MIF
    ind = ind + 1
print(14)     
######################################################################################################   
ind = 0 
for i in student15.index:
    MIF=0
    for col in student15.columns :
        if type(student15[col][i]) == np.float64 or type(student15[col][i]) == float:
            if any(student15[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student15[col][0:i+1].values))
                #print(col, MIF)
                
    student15.iloc[ind,-1] = MIF
    ind = ind + 1
print(15)     
######################################################################################################   
ind = 0 
for i in student16.index:
    MIF=0
    for col in student16.columns :
        if type(student16[col][i]) == np.float64 or type(student16[col][i]) == float:
            if any(student16[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student16[col][0:i+1].values))
                #print(col, MIF)
                
    student16.iloc[ind,-1] = MIF
    ind = ind + 1
print(16)     
######################################################################################################   
ind = 0 
for i in student17.index:
    MIF=0
    for col in student17.columns :
        if type(student17[col][i]) == np.float64 or type(student17[col][i]) == float:
            if any(student17[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student17[col][0:i+1].values))
                #print(col, MIF)
                
    student17.iloc[ind,-1] = MIF
    ind = ind + 1
print(17)     
######################################################################################################   
ind = 0 
for i in student18.index:
    MIF=0
    for col in student18.columns :
        if type(student18[col][i]) == np.float64 or type(student18[col][i]) == float:
            if any(student18[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student18[col][0:i+1].values))
                #print(col, MIF)
                
    student18.iloc[ind,-1] = MIF
    ind = ind + 1
print(18) 
######################################################################################################   
ind = 0 
for i in student19.index:
    MIF=0
    for col in student19.columns :
        if type(student19[col][i]) == np.float64 or type(student19[col][i]) == float:
            if any(student19[col][0:ind+1].notnull()): 
                MIF = MIF + int(np.nanmax(student19[col][0:i+1].values))
                #print(col, MIF)
                
    student19.iloc[ind,-1] = MIF
    ind = ind + 1
print(19) 


/opt/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19


In [14]:
#individually assign total MIFS
TM1 = np.array(student1["Total MIF"])
TM2 = np.array(student2["Total MIF"])
TM3 = np.array(student3["Total MIF"])
TM4 = np.array(student4["Total MIF"])
TM5 = np.array(student5["Total MIF"])
TM6 = np.array(student6["Total MIF"])
TM7 = np.array(student7["Total MIF"])
TM8 = np.array(student8["Total MIF"])
TM9 = np.array(student9["Total MIF"])
TM10 = np.array(student10["Total MIF"])
TM11 = np.array(student11["Total MIF"])
TM12 = np.array(student12["Total MIF"])
TM13 = np.array(student13["Total MIF"])
TM14 = np.array(student14["Total MIF"])
TM15 = np.array(student15["Total MIF"])
TM16 = np.array(student16["Total MIF"])
TM17 = np.array(student17["Total MIF"])
TM18 = np.array(student18["Total MIF"])
TM19 = np.array(student19["Total MIF"])

In [15]:
#combine total mifs in correct order order and add to main dataset
# ALLTotalMIF = np.concatenate((TM1, TM10, TM11, TM12, TM13, TM14, TM15, TM16, TM17, TM18, TM19,
#        TM2, TM3, TM4, TM5, TM6, TM7, TM8, TM9))

ALLTotalMIF = np.concatenate((TM1, TM2, TM3, TM4, TM5, TM6, TM7, TM8, TM9, TM10, TM11,
       TM12, TM13, TM14, TM15, TM16, TM17, TM18, TM19))

PTNdata1['Total MIF'] = ALLTotalMIF


### INSIGHT: 
It looks like student 6 failed out. The number of training days is less than half of the others and when I look at their grades, they get U or F on most maneuvers.

# Excluding Student 6
Drop student 6 from the dataset

In [16]:
PTNdata1[PTNdata1['Student ID']==6]
PTNdata1 = PTNdata1.drop(PTNdata1[PTNdata1['Student ID']==6].index, axis = 0)

#replace empty values and NG with value 0. 
PTNdata1.iloc[:,:-3] = PTNdata1.iloc[:,:-3].replace([np.nan,'',' ','NG','U','F','G','E'], [np.float64(0),np.float64(0),np.float64(0), np.float64(0), np.float64(1),np.float64(2),np.float64(3),np.float64(4)])

#PTNdata1.head(200)

# Export Cleaned data to CSV file

In [17]:
#Export cleaned data to CSV file 
PTNdata1.to_csv(r'/Users/nickcforrest/Documents/My Documents/Education/AFIT/AFIT WORK/Thesis Work/Python Files/PTNdata1_clean.csv')